# Tutorial

## Installing everything

To install a backend of choice (for example InteractUIkit), simply type
```julia
Pkg.clone("https://github.com/piever/InteractBase.jl")
Pkg.clone("https://github.com/piever/InteractUIkit.jl")
Pkg.build("InteractUIkit");
```

in the REPL.

The basic behavior is as follows: Interact provides a series of widgets, each widgets has a primary observable that can be obtained with `observe(widget)` and adding listeners to that observable one can provide behavior. Let's see this in practice.

## Displaying a widget

In [1]:
using InteractBulma, WebIO
ui = button()
display(ui)

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Recompiling stale cache file /home/pietro/.julia/lib/v0.6/InteractBase.ji for module InteractBase.
INFO: Recompiling stale cache file /home/pietro/.julia/lib/v0.6/InteractBulma.ji for module InteractBulma.


(div { className="interactbulma field" }
    (button { attributes=Dict{Any,Any}(Pair{Any,Any}("class", "interactbulma button is-primary"),Pair{Any,Any}("v-on:click", "clicks += 1")) }
    "Press me!"))

Note that `display` works in a [Jupyter notebook](https://github.com/JuliaLang/IJulia.jl) or in [Atom/Juno IDE](https://github.com/JunoLab/Juno.jl).
InteractBase can also be deployed in Jupyter Lab, but that requires installing an extension first:

In [ ]:
cd(Pkg.dir("WebIO", "assets"))
;jupyter labextension install webio
;jupyter labextension enable webio/jupyterlab_entry

To deploy the app as a standalone Electron window, one would use [Blink.jl](https://github.com/JunoLab/Blink.jl):

In [ ]:
using Blink
w = Window()
body!(w, ui);

The app can also be served in a webpage via [Mux.jl](https://github.com/JuliaWeb/Mux.jl):

In [ ]:
using Mux
webio_serve(page("/", req -> ui), rand(8000:9000)) # serve on a random port

## Adding behavior
For now this button doesn't do anything. This can be changed by adding callbacks to its primary observable:

In [2]:
o = observe(ui)

Observables.Observable{Int64}("ob_01", 2, Any[WebIO.Backedge(WebIO.Scope("vue-component-aec28399-f27c-48ed-8836-075156407365", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :button), Any["Press me!"], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}("class", "interactbulma button is-primary"),Pair{Any,Any}("v-on:click", "clicks += 1")))), 1), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("clicks", (Observables.Observable{Int64}(#= circular reference @-7 =#), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractBulma/all.js", "/pkg/InteractBulma/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("(function (Vue){console.log((\"initialising \"+\"vue-component-aec28399-f27c-48ed-8836-075156407365\")); var options={\"data\":{\"clicks\":0},\"obskey\":\"clicks\"}; options.el=this.dom; var self=this; function init

Each observable holds a value and its value can be inspected with the `[]` syntax:

In [3]:
o[]

2

In the case of the button, the observable represents the number of times it has been clicked: click on it and check the value again.

To add some behavior to the widget we can use the `on` construct. `on` takes two arguments, a function and an observable. As soon as the observable is changed, the function is called with the latest value.

In [ ]:
on(println, o)

If you click again on the button you will see it printing the number of times it has been clicked so far.

*Tip*: anonymous function are very useful in this programming paradigm. For example, if you want the button to say "Hello!" when pressed, you should use:

In [ ]:
on(n -> println("Hello!"), o)

*Tip n. 2*: using the `[]` syntax you can also set the value of the observable:

In [ ]:
o[] = 33;

To learn more about Observables, check out their documentation [here](https://juliagizmos.github.io/Observables.jl/latest/).
## What widgets are there?

Once you have grasped this paradigm, you can play with any of the many widgets available:

In [4]:
filepicker() |> display # observable is the path of selected file
textbox("Write here") |> display # observable is the text typed in by the user
autocomplete(["Mary", "Jane", "Jack"]) |> display # as above, but you can autocomplete words
checkbox(label = "Check me!") |> display # observable is a boolean describing whether it's ticked
toggle(label = "I have read and agreed") |> display # same as a checkbox but styled differently
slider(1:100, label = "To what extent?", value = 33) |> display # Observable is the number selected

(div { className="interactbulma field" }
  (div { className="interactbulma field" }
        (div { className="file" }
      (label { className="file-label" }
        (input { attributes=Dict{Any,Any}(Pair{Any,Any}("class", "interactbulma input interactbulma file-input "),Pair{Any,Any}("type", "file"),Pair{Any,Any}("v-on:change", "onFileChange"),Pair{Any,Any}("ref", "data")) })
        (span { className="file-cta" }
          (span { className="file-icon" }
            (i { className="fas fa-upload" }))
          (span { className="file-label" }
            "Choose a file..."))
        (span { className="file-name" }
          "{{filename == '' ? 'No file chosen' : filename}}")))))

(div { className="interactbulma field" }
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:type, "text"),Pair{Any,Any}(Symbol("v-model"), "internalvalue"),Pair{Any,Any}(:placeholder, "Write here")) className="interactbulma input " }))

(div { className="interactbulma field" }
    (div
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:list, Symbol("##661")),Pair{Any,Any}(:type, "text"),Pair{Any,Any}(Symbol("v-model"), "internalvalue"),Pair{Any,Any}(:placeholder, "")) className="interactbulma input " })
    (datalist { attributes=Dict("id"=>"##661") }
      (option { attributes=Dict("value"=>"Mary") })
      (option { attributes=Dict("value"=>"Jane") })
      (option { attributes=Dict("value"=>"Jack") }))))

(div { className="interactbulma field" }
    (div { className="field" }
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:id, "##662"),Pair{Any,Any}(:type, "checkbox"),Pair{Any,Any}(Symbol("v-model.number"), "internalvalue")) className="interactbulma is-checkradio " })
    (label { attributes=Dict("class"=>"","for"=>"##662") }
      "Check me!")))

(div { className="interactbulma field" }
    (div { className="field" }
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:id, "##663"),Pair{Any,Any}(:type, "checkbox"),Pair{Any,Any}(Symbol("v-model.number"), "internalvalue")) className="interactbulma switch " })
    (label { attributes=Dict("class"=>"","for"=>"##663") }
      "I have read and agreed")))

(div { className="interactbulma field" }
  (div { className="interactbulma field" }
        (div { attributes=Dict("style"=>"display:flex; justify-content:center; align-items:center;") }
      (div { attributes=Dict("style"=>"text-align:right;width:18%") }
        (label { className="interactbulma " style=Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")) }
          "To what extent?"))
      (div { attributes=Dict("style"=>"width:60%; margin: 0 2%") }
        (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:max, 100),Pair{Any,Any}(:step, 1),Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("v-model.number"), "internalvalue"),Pair{Any,Any}(:min, 1)) className="interactbulma slider is-fullwidth" }))
      (div { attributes=Dict("style"=>"width:18%") }
        (div
          "{{displayedvalue}}")))))

As well as the option widgets, that allow to choose among options:

In [7]:
dropdown(["a", "b", "c"]) |> display # Observable is option selected
togglebuttons(["a", "b", "c"]) |> dom"div.field.interactbulma" |> display # Observable is option selected
radiobuttons(["a", "b", "c"]) |> display # Observable is option selected

(div { className="interactbulma field" }
    (div { className="select" }
    (select { attributes=Dict{Symbol,Any}(Pair{Symbol,Any}(Symbol("v-model"), "value")) className="interactbulma input " }
      (option { attributes=Dict(:value=>"a") }
        "a")
      (option { attributes=Dict(:value=>"b") }
        "b")
      (option { attributes=Dict(:value=>"c") }
        "c"))))

(div { className="field interactbulma" }
  InteractBase.Widget{:togglebuttons}(Val{:togglebuttons}(), WebIO.Scope("vue-component-848bc760-4c0d-4cef-b96a-884f8623535d", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["a"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 1),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 1}]"),Pair{String,Any}("v-on:click", "changeValue(1)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["b"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 2),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 2}]"),Pair{String,Any}("v-on:click", "changeValue(2)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["c"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 3),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 3}]"),Pair{String,Any}("v-on:click", "changeValue(3)")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{Any,Any}()),Pair{Symbol,Any}(:className, "interactbulma field is-grouped has-addons is-oneline is-centered")), 6), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("index", (Observables.Observable{Int64}("ob_18", 2, Any[Observables.g, WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractBulma/all.js", "/pkg/InteractBulma/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("(function (Vue){console.log((\"initialising \"+\"vue-component-848bc760-4c0d-4cef-b96a-884f8623535d\")); var options={\"methods\":{\"changeValue\":function (num){\n    return this.index = num;\n}\n},\"data\":{\"index\":2}}; options.el=this.dom; var self=this; function init(){this.vue=new Vue(options); return this.vue[\"\$watch\"](\"index\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"index\",\"scope\":\"vue-component-848bc760-4c0d-4cef-b96a-884f8623535d\",\"id\":\"ob_18\",\"type\":\"observable\"},valcopy)}))}; return setTimeout((function (){return init.call(self)}),0)})")]))),Pair{Any,Any}("index", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); return this.vue[\"index\"]=valcopy})")])), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set{WebIO.AbstractConnection}(), Channel{WebIO.AbstractConnection}(sz_max:32,sz_curr:0))), WebIO.Scope("vue-component-848bc760-4c0d-4cef-b96a-884f8623535d", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["a"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 1),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 1}]"),Pair{String,Any}("v-on:click", "changeValue(1)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["b"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 2),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 2}]"),Pair{String,Any}("v-on:click", "changeValue(2)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["c"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 3),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-

(div { className="interactbulma field" }
    (div { attributes=Dict{Any,Any}() className="field" }
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:value, "a"),Pair{Any,Any}(:id, "##680"),Pair{Any,Any}(:type, "radio"),Pair{Any,Any}(:name, Symbol("##679")),Pair{Any,Any}("v-model", "value")) className="is-checkradio " })
    (label { attributes=Dict{Any,Any}(Pair{Any,Any}(:for, "##680")) className="form-radio" }
      "a")
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:value, "b"),Pair{Any,Any}(:id, "##681"),Pair{Any,Any}(:type, "radio"),Pair{Any,Any}(:name, Symbol("##679")),Pair{Any,Any}("v-model", "value")) className="is-checkradio " })
    (label { attributes=Dict{Any,Any}(Pair{Any,Any}(:for, "##681")) className="form-radio" }
      "b")
    (input { attributes=Dict{Any,Any}(Pair{Any,Any}(:value, "c"),Pair{Any,Any}(:id, "##682"),Pair{Any,Any}(:type, "radio"),Pair{Any,Any}(:name, Symbol("##679")),Pair{Any,Any}("v-model", "value")) className="is-checkradio " })
    (label { attributes=Dict{Any,Any}(Pair{Any,Any}(:for, "##682")) className="form-radio" }
      "c")))

The option widgets can also take as input a dictionary (ordered dictionary is preferrable, to avoid items getting scrambled), in which case the label displays the key while the observable stores the value:

In [ ]:
using DataStructures
s = dropdown(OrderedDict("a" => "Value 1", "b" => "Value 2"))
display(s)

In [ ]:
observe(s)[]

## A simpler approach for simpler cases

While the approach sketched above works for all sorts of situations, there is a specific marcro to simplify it in some specific case. If you want to update some result (maybe a plot) as a function of some parameters (discrete or continuous) simply write `@manipulate` before the `for` loop. Discrete parameters will be replaced by `togglebuttons` and continuous parameters by `slider`: the result will be updated as soon as you click on a button or move the slider:

In [8]:
width, height = 700, 300
colors = ["black", "gray", "silver", "maroon", "red", "olive", "yellow", "green", "lime", "teal", "aqua", "navy", "blue", "purple", "fuchsia"]
color(i) = colors[i%length(colors)+1]
ui = @manipulate for nsamples in 1:200,
        sample_step in slider(0.01:0.01:1.0, value=0.1, label="sample step"),
        phase in slider(0:0.1:2pi, value=0.0, label="phase"),
        radii in 0.1:0.1:60
    cxs_unscaled = [i*sample_step + phase for i in 1:nsamples]
    cys = sin.(cxs_unscaled) .* height/3 .+ height/2
    cxs = cxs_unscaled .* width/4pi
    dom"svg:svg[width=$width, height=$height]"(
        (dom"svg:circle[cx=$(cxs[i]), cy=$(cys[i]), r=$radii, fill=$(color(i))]"()
            for i in 1:nsamples)...
    )
end

(div { className="interactbulma" }
  InteractBase.Widget{:slider}(Val{:slider}(), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope("vue-component-82a4bd45-296f-4d4e-bb74-af6b28eaaa26", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["nsamples"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interactbulma "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}(:max, 200),Pair{Any,Any}(:step, 1),Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("v-model.number"), "internalvalue"),Pair{Any,Any}(:min, 1))),Pair{Any,Any}(:className, "interactbulma slider is-fullwidth")), 0)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"width:60%; margin: 0 2%"))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any["{{displayedvalue}}"], Dict{Any,Any}(), 1)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"width:18%"))), 2)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 8), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("internalvalue", (Observables.Observable{Int64}("ob_21", 100, Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("value", (Observables.Observable{Int64}("ob_21", 100, Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractBulma/all.js", "/pkg/InteractBulma/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("(function (Vue){console.log((\"initialising \"+\"vue-component-82a4bd45-296f-4d4e-bb74-af6b28eaaa26\")); var options={\"computed\":{\"displayedvalue\":function () {return this.value;}},\"data\":{\"internalvalue\":100,\"value\":100}}; options.el=this.dom; var self=this; function init(){this.vue=new Vue(options); return this.vue[\"\$watch\"](\"internalvalue\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"internalvalue\",\"scope\":\"vue-component-82a4bd45-296f-4d4e-bb74-af6b28eaaa26\",\"id\":\"ob_21\",\"type\":\"observable\"},valcopy)})),this.vue[\"\$watch\"](\"value\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"value\",\"scope\":\"vue-component-82a4bd45-296f-4d4e-bb74-af6b28eaaa26\",\"id\":\"ob_21\",\"type\":\"observable\"},valcopy)}))}; return setTimeout((function (){return init.call(self)}),0)})")]))),Pair{Any,Any}("internalvalue", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); return this.vue[\"internalvalue\"]=valcopy})")]),Pair{Any,Any}("value", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); return this.vue[\"value\"]=valcopy})")])), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:2), Set{WebIO.AbstractConnection}(), Channel{WebIO.AbstractConnection}(sz_max:32,sz_curr:0)))], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interactbulma field")), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interactbulma field")), 2), WebIO.Scope("vue-component-82a4bd45-296f-4d4e-bb74-af6b28eaaa26", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Nod

or, if you want a plot with some variables taking discrete values:

In [9]:
using Plots, DataStructures

x = y = 0:0.1:30

freqs = OrderedDict(zip(["pi/4", "π/2", "3π/4", "π"], [π/4, π/2, 3π/4, π]))

mp = @manipulate for freq1 in freqs, freq2 in slider(0.01:0.1:4π; label="freq2")
    y = @. sin(freq1*x) * sin(freq2*x)
    plot(x, y)
end

(div { className="interactbulma" }
  InteractBase.Widget{:togglebuttons}(Val{:togglebuttons}(), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope("vue-component-bb4adcdc-acf6-4b0f-911b-8916d0f529be", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["freq1"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interactbulma "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["pi/4"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 1),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 1}]"),Pair{String,Any}("v-on:click", "changeValue(1)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["π/2"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 2),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 2}]"),Pair{String,Any}("v-on:click", "changeValue(2)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["3π/4"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 3),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 3}]"),Pair{String,Any}("v-on:click", "changeValue(3)")))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["π"], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("key", 4),Pair{String,Any}("v-bind:class", "['interactbulma input button is-fullwidth', {'interactbulma input is-primary is-selected' : index == 4}]"),Pair{String,Any}("v-on:click", "changeValue(4)")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{Any,Any}()),Pair{Symbol,Any}(:className, "interactbulma field is-grouped has-addons is-oneline is-centered")), 8)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"width:60%; margin: 0 2%"))), 9), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Any,Any}(), 0)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"width:18%"))), 1)], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 15), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("index", (Observables.Observable{Int64}("ob_27", 2, Any[Observables.g, WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractBulma/all.js", "/pkg/InteractBulma/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("(function (Vue){console.log((\"initialising \"+\"vue-component-bb4adcdc-acf6-4b0f-911b-8916d0f529be\")); var options={\"methods\":{\"changeValue\":function (num){\n    return this.index = num;\n}\n},\"data\":{\"index\":2}}; options.el=this.dom; var self=this; function init(){this.vue=new Vue(options); return this.vue[\"\$watch\"](\"index\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"index\",\"scope\":\"vue-component-bb4adcdc-acf6-4b0f-911b-8916d0f529be\",\"id\":\"ob_27\",\"type\":\"observable\"},valcopy)}))}; return setTimeout((function (){return init.call(self)}),0)})")]))),Pair{Any,Any}("index", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); ret

## Layout

To create a full blown web-app, you should learn the layout tools that the CSS framework you are using provides. Both [Bulma](https://bulma.io/) and [UIkit](https://getuikit.com/) have modern layout tools for responsive design (of course, use Bulma if you're working with InteractBulma and UIkit if you're working with InteractUIkit). You can use [WebIO](https://github.com/JuliaGizmos/WebIO.jl) to create from Julia the HTML required to create these layouts.

However, this can be overwhelming at first (especially for users with no prior experience in web design). A simpler solution is [CSSUtil](https://github.com/JuliaGizmos/CSSUtil.jl), a package that provides some tools to create simple layouts.

In [ ]:
using CSSUtil
loadbutton = filepicker()
hellobutton = button("Hello!")
goodbyebutton = button("Good bye!")
ui = vbox( # put things one on top of the other
    loadbutton,
    hbox( # put things one next to the other
        pad(1em, hellobutton), # to allow some white space around the widget
        pad(1em, goodbyebutton),
    )
)
display(ui)

## Update widgets as function of other widgets

Sometimes the full structure of the GUI is not known in advance. For example, let's imagine we want to load a DataFrame and create a button per column. Not to make it completely trivial, as soon as a button is pressed, we want to plot a histogram of the corresponding column.

*Important note*: this app needs to run in Blink, as the browser doesn't allow us to get access to the local path of a file.

We start by adding a `filepicker` to choose the file, and only once we have a file we want to update the GUI. this can be done as follows:

In [ ]:
loadbutton = filepicker()
columnbuttons = Observable{Any}(dom"div"())

`columnbuttons` is the `div` object that will contain all the relevant buttons. it is an `Observable` as we want its value to change over time.
To add behavior, we can use `map!`:

In [ ]:
using CSV, DataFrames
data = Observable{Any}(DataFrame)
map!(CSV.read, data, observe(loadbutton))

Now as soon as a file is uploaded, the `Observable` `data` gets updated with the correct value. Now, as soon as `data` is updated, we want to update our buttons.

In [ ]:
using CSSUtil
function makebuttons(df)
    buttons = button.(names(df))
    dom"div"(hbox(buttons))
end

map!(makebuttons, columnbuttons, data)

Note that `data` is already an `Observable`, so there's no need to do `observe(data)`, `observe` can only be applied on a widget.
We are almost done, we only need to add a callback to the buttons. The cleanest way is to do it during button initialization, meaning during our `makebuttons` step:

In [ ]:
using Plots
plt = Observable{Any}(plot()) # the container for our plot
function makebuttons(df)
    buttons = button.(string.(names(df)))
    for (btn, name) in zip(buttons, names(df))
        map!(t -> histogram(df[name]), plt, observe(btn))
    end
    dom"div"(hbox(buttons))
end

To put it all together:

In [10]:
using CSV, DataFrames, InteractUIkit, WebIO, Observables, Plots, CSSUtil
loadbutton = filepicker()
columnbuttons = Observable{Any}(dom"div"())
data = Observable{Any}(DataFrame)
plt = Observable{Any}(plot())
map!(CSV.read, data, observe(loadbutton))

function makebuttons(df)
    buttons = button.(string.(names(df)))
    for (btn, name) in zip(buttons, names(df))
        map!(t -> histogram(df[name]), plt, observe(btn))
    end
    dom"div"(hbox(buttons))
end

map!(makebuttons, columnbuttons, data)

ui = dom"div"(loadbutton, columnbuttons, plt)

INFO: Recompiling stale cache file /home/pietro/.julia/lib/v0.6/InteractUIkit.ji for module InteractUIkit.


(div
  InteractBase.Widget{:filepicker}(Val{:filepicker}(), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope("vue-component-a8a86342-7a69-4aba-b95a-8f22d13bfc1b", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}("class", "uk-input "),Pair{Any,Any}("type", "file"),Pair{Any,Any}("v-on:change", "onFileChange"),Pair{Any,Any}("ref", "data")))), 0), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("filename", (Observables.Observable{String}("ob_33", "", Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("path", (Observables.Observable{String}("ob_32", "", Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractUIkit/uikit-icons.min.js", "/pkg/InteractUIkit/uikit.min.css", "/pkg/InteractUIkit/uikit.min.js", "/pkg/InteractUIkit/toggle.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("(function (Vue){console.log((\"initialising \"+\"vue-component-a8a86342-7a69-4aba-b95a-8f22d13bfc1b\")); var options={\"methods\":{\"onFileChange\":function (event){\n    this.filename = this.\$refs.data.files[0].name;\n    return this.path = this.\$refs.data.files[0].path;\n}\n},\"data\":{\"filename\":\"\",\"path\":\"\"}}; options.el=this.dom; var self=this; function init(){this.vue=new Vue(options); return this.vue[\"\$watch\"](\"filename\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"filename\",\"scope\":\"vue-component-a8a86342-7a69-4aba-b95a-8f22d13bfc1b\",\"id\":\"ob_33\",\"type\":\"observable\"},valcopy)})),this.vue[\"\$watch\"](\"path\",(function (val,oldval){var valcopy=JSON.parse(JSON.stringify(val)); return WebIO.setval({\"name\":\"path\",\"scope\":\"vue-component-a8a86342-7a69-4aba-b95a-8f22d13bfc1b\",\"id\":\"ob_32\",\"type\":\"observable\"},valcopy)}))}; return setTimeout((function (){return init.call(self)}),0)})")]))),Pair{Any,Any}("filename", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); return this.vue[\"filename\"]=valcopy})")]),Pair{Any,Any}("path", Any[WebIO.JSString("(function (val){var valcopy=JSON.parse(JSON.stringify(val)); return this.vue[\"path\"]=valcopy})")])), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:3), Set{WebIO.AbstractConnection}(), Channel{WebIO.AbstractConnection}(sz_max:32,sz_curr:0)))], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "uk-margin")), 1), WebIO.Scope("vue-component-a8a86342-7a69-4aba-b95a-8f22d13bfc1b", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Any,Any}(Pair{Any,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}("class", "uk-input "),Pair{Any,Any}("type", "file"),Pair{Any,Any}("v-on:change", "onFileChange"),Pair{Any,Any}("ref", "data")))), 0), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("filename", (Observables.Observable{String}("ob_33", "", Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37))]), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("path", (Observables.Observable{String}("ob_32", "", Any[WebIO.Backedge(WebIO.Scope(#= circular reference @-7 =#), WebIO.Backedge(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["vue"=>"/pkg/Vue/vue.js", "/pkg/InteractUIkit/uikit-icons.min.js", "/pkg/InteractUIkit/uikit.min.css", "/pkg/InteractUIkit/uikit.min.js", "/pkg/InteractUIkit/toggle.css"], Dict{An

And now to serve it in Blink:

In [11]:
using Blink
w = Window()
body!(w, ui)

Blink.Page(1, WebSockets.WebSocket(5, TCPSocket(RawFD(137) active, 0 bytes waiting), CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true)))

In [ ]:
vbox(button(), button())

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*